# RoBERTa for Problem 2

In [377]:
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer,TFRobertaModel

In [7]:
pip install --upgrade ipython

     ---------------------------------------- 0.0/797.7 kB ? eta -:--:--
     -------- ----------------------------- 174.1/797.7 kB 3.5 MB/s eta 0:00:01
     ------------------------- ------------ 542.7/797.7 kB 5.7 MB/s eta 0:00:01
     -------------------------------------- 797.7/797.7 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 8.11.0
    Uninstalling ipython-8.11.0:
      Successfully uninstalled ipython-8.11.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [387]:
df = pd.read_csv('all_11_models_including_human.csv')

In [388]:
# df = pd.read_csv('all_11_methods_without_cleaning_text.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [389]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [390]:
def get_roberta_embedding(sentence):
    tokens = tokenizer.encode(sentence, max_length=512, truncation=True)
    input_ids = tf.constant(tokens)[None, :]
    embedding = roberta_model(input_ids)[0][0]
    return embedding.numpy()

In [ ]:
train_embeddings = np.vstack(tqdm(train_df['text'].apply(get_roberta_embedding)))

In [ ]:
test_embeddings = np.vstack(tqdm.tqdm(test_df['text'].apply(get_roberta_embedding)))

In [ ]:
type(train_embeddings)

In [ ]:
classifier = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(11, activation='softmax')
])

In [ ]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                   metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# Train neural network classifier
classifier.fit(train_embeddings, train_df['class'], batch_size=32, epochs=10, validation_split=0.2)

# Task 2 

In [350]:
d1 = pd.read_csv(r"C:\Users\shiny\Downloads\Authorship-Attribution-for-Neural-Text-Generation-master (1)\Authorship-Attribution-for-Neural-Text-Generation-master\data\balanced_p2.csv")

In [351]:
d1 = d1.drop("Unnamed: 0",axis = 1)

In [352]:
d1

,text,class
0,bbc london the british government is expected ...,1
1,workers fight for rights at ground zero for us...,1
2,entrepreneurship and the arts the museum of mo...,1
3,in coronavirus crisis tom hanks is more of a r...,1
4,zunar and fahmi reza the cartoonists who helpe...,1
...,...,...
2127,How much of your body is your own? this story ...,0
2128,How do you keep a space station clean? by 1998...,0
2129,The city where you pay a year's rent up front ...,0
2130,The BBC News app gives you the best of BBC New...,0


In [353]:
T1 = d1["text"].to_frame().rename(columns={'text': 'Generation'})

In [ ]:
T1_embeddings = np.vstack(T1['Generation'].apply(get_roberta_embedding))

In [ ]:
y_pred_t1 = classifier.predict(T1_embeddings)

In [ ]:
y_pred_1 = np.argmax(y_pred_t1,axis = 1)

In [ ]:
t1_names = label_encoder.inverse_transform(y_pred_1)
actual = d1['class'].to_numpy()
predictions = []
for i in range(len(t1_names)):
    
    if t1_names[i] == 'human':
        predictions.append(0)
    else:
        predictions.append(1)
        
predictions = np.array(predictions)
print(actual)
print(predictions)

print(classification_report(actual,predictions))

# POS + LSTM-LSTM

In [15]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.Collecting spacy
     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
      --------------------------------------- 0.2/12.2 MB 3.3 MB/s eta 0:00:04
     - -------------------------------------- 0.5/12.2 MB 5.9 MB/s eta 0:00:02
     -- ------------------------------------- 0.8/12.2 MB 6.3 MB/s eta 0:00:02
     --- ------------------------------------ 1.0/12.2 MB 6.0 MB/s eta 0:00:02
     ---- ----------------------------------- 1.3/12.2 MB 5.9 MB/s eta 0:00:02
     ---- ----------------------------------- 1.5/12.2 MB 6.0 MB/s eta 0:00:02
     ----- ---------------------------------- 1.7/12.2 MB 5.5 MB/s eta 0:00:02
     ------ --------------------------------- 2.0/12.2 MB 5.6 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.2 MB 5.7 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.2 MB 5.5 MB/s eta 0:00:02
     -------- ------------------------------- 2.5/12.2 M


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [287]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GRU

In [19]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.2/12.8 MB 7.3 MB/s eta 0:00:02
     -- ------------------------------------- 0.7/12.8 MB 8.7 MB/s eta 0:00:02
     --- ------------------------------------ 1.0/12.8 MB 8.1 MB/s eta 0:00:02
     ---- ----------------------------------- 1.4/12.8 MB 7.8 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 8.2 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 8.7 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 8.8 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 8.7 MB/s eta 0:00:02
     ----------- ---------------------------- 3.6/12.8 MB 9.0 MB/s eta 0:00:02
     ------------ --------------------------- 4.0/12.8 MB 9.2 MB/s eta 0:00:01
     ------------ --------------------------- 4.0/12.8 MB 8.9 MB/s eta 0:00:01
     ------------ --------------------------- 4.0/12.8 MB 8


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
nlp = spacy.load('en_core_web_sm')

In [21]:
df = pd.read_csv('all_11_methods_without_cleaning_text.csv')

In [24]:
df = df.drop("Unnamed: 0",axis=1)
df

,Generation,label
0,['News Latest Headlines on CNN Business \n TL;...,ctrl
1,"[""News China wants to take a victory lap over ...",ctrl
2,"[""News Coronavirus disinformation creates chal...",ctrl
3,"[""News China coronavirus: Eating wild animals ...",ctrl
4,"[""News China's economy could shrink for the fi...",ctrl
...,...,...
11721,"""How much of your body is your own?"""""" """" ""All...",instructgpt
11722,"""How do you keep a space station clean?"""""" """" ...",instructgpt
11723,"""The city where you pay a year's rent up front...",instructgpt
11724,"""The BBC News app gives you the best of BBC Ne...",instructgpt


In [29]:
def preprocess(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9\s]', '',text)
    return text

In [33]:
def extract_features(text):
    doc = nlp(text)
    pos_tags = [token.pos_ for token in doc]
    named_entities = [ent.label_ for ent in doc.ents]
    dep_tags = [token.dep_ for token in doc]
    features = pos_tags + named_entities + dep_tags
    return features

In [34]:
df['Generation'] = df['Generation'].apply(preprocess)
df['Features'] = df['Generation'].apply(extract_features)

In [35]:
df['Features'] = df['Features'].apply(lambda x: ' '.join(x))

In [190]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(df['Features'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [191]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['label'])

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [193]:
max_len = X_train_tfidf.shape[1]
input_shape = (max_len,)

In [197]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=input_shape),
    tf.keras.layers.Reshape((max_len, 1)),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

In [198]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [199]:
model.fit(X_train.toarray(), y_train, epochs=60, batch_size=32, validation_data=(X_val.toarray(), y_val))

Epoch 1/60
235/235 [==============================] - 11s 35ms/step - loss: 2.2118 - accuracy: 0.1890 - val_loss: 1.8671 - val_accuracy: 0.3230
Epoch 2/60
235/235 [==============================] - 8s 33ms/step - loss: 1.6438 - accuracy: 0.3910 - val_loss: 1.5662 - val_accuracy: 0.4483
Epoch 3/60
235/235 [==============================] - 8s 32ms/step - loss: 1.5187 - accuracy: 0.4443 - val_loss: 1.5921 - val_accuracy: 0.4286
Epoch 4/60
235/235 [==============================] - 8s 32ms/step - loss: 1.4749 - accuracy: 0.4570 - val_loss: 1.4679 - val_accuracy: 0.4728
Epoch 5/60
235/235 [==============================] - 8s 32ms/step - loss: 1.4236 - accuracy: 0.4775 - val_loss: 1.4009 - val_accuracy: 0.4845
Epoch 6/60
235/235 [==============================] - 8s 33ms/step - loss: 1.3513 - accuracy: 0.5055 - val_loss: 1.4075 - val_accuracy: 0.4851
Epoch 7/60
235/235 [==============================] - 8s 33ms/step - loss: 1.3209 - accuracy: 0.5201 - val_loss: 1.3951 - val_accuracy: 0.479

Epoch 58/60
235/235 [==============================] - 8s 32ms/step - loss: 0.6083 - accuracy: 0.7705 - val_loss: 0.8437 - val_accuracy: 0.7036
Epoch 59/60
235/235 [==============================] - 7s 32ms/step - loss: 0.6148 - accuracy: 0.7684 - val_loss: 0.8060 - val_accuracy: 0.7095
Epoch 60/60
235/235 [==============================] - 8s 32ms/step - loss: 0.5820 - accuracy: 0.7856 - val_loss: 0.8776 - val_accuracy: 0.6892


In [200]:
y_pred = model.predict(X_test.toarray())

74/74 [==============================] - 1s 11ms/step


In [201]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([4, 8, 4, ..., 3, 0, 4], dtype=int64)

In [202]:
y_test

array([ 6,  5,  4, ...,  3, 10,  4])

In [203]:
print(confusion_matrix(y_test, y_pred))

[[210   1   0   0   0   0   0   0   0   0   3]
 [  0  88   1  30   7  53   6  10  13   1   0]
 [ 11   0 201   0   1   0   3   0   1   0   0]
 [  0  48   0 124  10  14   1  11   3   1   1]
 [  0   5   3   2 111   3  21  41  28   1   2]
 [  1  44   0   6   2 137   7  16   6   0   3]
 [  1   3   3   0  32   3 138   9  29   1   1]
 [  1  10   0  11  29   8  13 101  26   3   0]
 [  0   0   0   0  17   1  14  10 164   0   0]
 [  4   0   1   0   0   0   0   2   0 195  13]
 [ 36   0   5   0   2   0   0   1   1   1 165]]


In [204]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

        ctrl       0.80      0.98      0.88       214
        fair       0.44      0.42      0.43       209
         gpt       0.94      0.93      0.93       217
        gpt2       0.72      0.58      0.64       213
        gpt3       0.53      0.51      0.52       217
      grover       0.63      0.62      0.62       222
       human       0.68      0.63      0.65       220
 instructgpt       0.50      0.50      0.50       202
        pplm       0.61      0.80      0.69       206
         xlm       0.96      0.91      0.93       215
       xlnet       0.88      0.78      0.83       211

    accuracy                           0.70      2346
   macro avg       0.70      0.70      0.69      2346
weighted avg       0.70      0.70      0.69      2346



In [205]:
model.save('POS_LSTM_LSTM.h5')

# Task - 3

In [250]:
d2 = pd.read_csv(r"C:\Users\shiny\Downloads\Authorship-Attribution-for-Neural-Text-Generation-master (1)\Authorship-Attribution-for-Neural-Text-Generation-master\Clean_Generated\task_3_data_without_cleaning.csv")

In [251]:
# d2 = d2.drop("Unnamed: 0",axis = 1)

In [252]:
d2['label'].value_counts()

ctrl           500
gpt            500
gpt2           500
grover         500
xlm            500
xlnet          500
pplm           500
fair           500
gpt3           500
instructgpt    500
Name: label, dtype: int64

In [253]:
T1 = d2["Generation"].to_frame()

In [254]:
T1

,Generation
0,['News Latest Headlines on CNN Business \n TL;...
1,"[""News China wants to take a victory lap over ..."
2,"[""News Coronavirus disinformation creates chal..."
3,"[""News China coronavirus: Eating wild animals ..."
4,"[""News China's economy could shrink for the fi..."
...,...
4995,"""One dead in suicide bomb attack near US embas..."
4996,"""Nigeria says it is ready and more than capabl..."
4997,"""Lesotho Prime Minister in surprise court appe..."
4998,"""Dutch government returns stolen 18th-century ..."


In [256]:
T1['Generation'] = T1['Generation'].apply(preprocess)
T1['Features'] = T1['Generation'].apply(extract_features)
T1['Features'] = T1['Features'].apply(lambda x: ' '.join(x))

In [257]:
T1

,Generation,Features
0,news latest headlines on cnn business n tldr t...,NOUN ADJ NOUN ADP PROPN NOUN VERB VERB DET PRO...
1,news china wants to take a victory lap over it...,PROPN PROPN VERB PART VERB DET NOUN NOUN ADP P...
2,news coronavirus disinformation creates challe...,NOUN NOUN NOUN VERB NOUN ADP PROPN PROPN PROPN...
3,news china coronavirus eating wild animals mad...,NOUN PROPN PROPN VERB ADJ NOUN VERB ADJ CCONJ ...
4,news chinas economy could shrink for the first...,NOUN PROPN PROPN AUX VERB ADP DET ADJ NOUN ADP...
...,...,...
4995,one dead in suicide bomb attack near us embass...,NUM ADJ ADP NOUN NOUN NOUN ADP PROPN NOUN ADP ...
4996,nigeria says it is ready and more than capable...,PROPN VERB PRON AUX ADJ CCONJ ADJ ADP ADJ ADP ...
4997,lesotho prime minister in surprise court appea...,PROPN PROPN PROPN ADP NOUN NOUN NOUN SCONJ NOU...
4998,dutch government returns stolen 18thcentury pr...,ADJ NOUN NOUN VERB NUM ADJ NOUN ADP NOUN SPACE...


In [258]:
X_test = vectorizer.transform(T1['Features'])
X_test = tfidf_transformer.transform(X_test)

In [260]:
loaded_model = tf.keras.models.load_model('POS_LSTM_LSTM.h5')
y_pred = loaded_model.predict(X_test.toarray())
y_pred = np.argmax(y_pred, axis=1)
y_pred

157/157 [==============================] - 3s 11ms/step


array([ 0,  0,  0, ...,  7,  8, 10], dtype=int64)

In [261]:
y_pred = le.inverse_transform(y_pred)

In [262]:
actual = d2['label'].to_numpy()

In [263]:
print(actual)
print(y_pred)

['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'instructgpt' 'instructgpt']
['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'pplm' 'xlnet']


In [265]:
print(classification_report(actual, y_pred, labels=['ctrl','fair','gpt','gpt2','grover','instructgpt','pplm','xlm','xlnet','gpt3', 'instructgpt']))

              precision    recall  f1-score   support

        ctrl       0.80      0.98      0.88       500
        fair       0.51      0.45      0.48       500
         gpt       0.96      0.93      0.95       500
        gpt2       0.78      0.61      0.69       500
      grover       0.64      0.70      0.67       500
 instructgpt       0.63      0.61      0.62       500
        pplm       0.72      0.85      0.78       500
         xlm       0.98      0.93      0.95       500
       xlnet       0.92      0.80      0.85       500
        gpt3       0.72      0.59      0.65       500
 instructgpt       0.63      0.61      0.62       500

   micro avg       0.75      0.73      0.74      5500
   macro avg       0.75      0.73      0.74      5500
weighted avg       0.75      0.73      0.74      5500



# POS + Stacked CNN

In [255]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(df['Features'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [118]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['label'])

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [120]:
max_len = X_train_tfidf.shape[1]
input_shape = (max_len,)

In [126]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=input_shape),
    tf.keras.layers.Reshape((max_len, 1)),
    tf.keras.layers.Conv1D(64, kernel_size=3, padding="valid", activation="relu"),
    tf.keras.layers.Conv1D(64, kernel_size=4, padding="valid", activation="relu"),
    tf.keras.layers.Conv1D(64, kernel_size=5, padding="valid", activation="relu"),
    tf.keras.layers.Conv1D(64, kernel_size=6, padding="valid", activation="relu"),
    tf.keras.layers.Conv1D(64, kernel_size=7, padding="valid", activation="relu"),
    tf.keras.layers.Conv1D(64, kernel_size=8, padding="valid", activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

In [127]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [129]:
model.fit(X_train.toarray(), y_train, epochs=15, batch_size=32, validation_data=(X_val.toarray(), y_val))

Epoch 1/15
235/235 [==============================] - 10s 32ms/step - loss: 1.7446 - accuracy: 0.3755 - val_loss: 1.3748 - val_accuracy: 0.4963
Epoch 2/15
235/235 [==============================] - 7s 31ms/step - loss: 1.0119 - accuracy: 0.6307 - val_loss: 0.9602 - val_accuracy: 0.6509
Epoch 3/15
235/235 [==============================] - 7s 31ms/step - loss: 0.8478 - accuracy: 0.6871 - val_loss: 0.8047 - val_accuracy: 0.6994
Epoch 4/15
235/235 [==============================] - 7s 30ms/step - loss: 0.7733 - accuracy: 0.7170 - val_loss: 0.7347 - val_accuracy: 0.7287
Epoch 5/15
235/235 [==============================] - 9s 37ms/step - loss: 0.7099 - accuracy: 0.7328 - val_loss: 0.6903 - val_accuracy: 0.7473
Epoch 6/15
235/235 [==============================] - 8s 34ms/step - loss: 0.6529 - accuracy: 0.7611 - val_loss: 0.6691 - val_accuracy: 0.7479
Epoch 7/15
235/235 [==============================] - 7s 31ms/step - loss: 0.6349 - accuracy: 0.7629 - val_loss: 0.7262 - val_accuracy: 0.711

In [130]:
y_pred = model.predict(X_test.toarray())

74/74 [==============================] - 1s 11ms/step


In [131]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([ 4,  3,  4, ...,  3, 10,  4], dtype=int64)

In [132]:
y_test

array([ 6,  5,  4, ...,  3, 10,  4])

In [133]:
print(confusion_matrix(y_test, y_pred))

[[211   2   0   0   0   0   0   0   0   0   1]
 [  0  75   0  63   7  53   1   9   1   0   0]
 [  1   0 208   0   3   2   2   1   0   0   0]
 [  0  21   0 170   1   9   6   6   0   0   0]
 [  0   0   0   3 133   1  20  57   3   0   0]
 [  0  17   0  27   1 167   0   5   3   0   2]
 [  0   9   1   3  33  25 121   9  19   0   0]
 [  1   3   0  17  43   5   3 126   4   0   0]
 [  0  10   0   3  23  10   3  14 143   0   0]
 [  1   1   0   5   0   0   1   1   2 204   0]
 [  9   1   0   2   0   1   0   0   1   0 197]]


In [134]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

        ctrl       0.95      0.99      0.97       214
        fair       0.54      0.36      0.43       209
         gpt       1.00      0.96      0.98       217
        gpt2       0.58      0.80      0.67       213
        gpt3       0.55      0.61      0.58       217
      grover       0.61      0.75      0.67       222
       human       0.77      0.55      0.64       220
 instructgpt       0.55      0.62      0.59       202
        pplm       0.81      0.69      0.75       206
         xlm       1.00      0.95      0.97       215
       xlnet       0.98      0.93      0.96       211

    accuracy                           0.75      2346
   macro avg       0.76      0.75      0.75      2346
weighted avg       0.76      0.75      0.75      2346



In [135]:
model.save('POS_Stacked_CNN.h5')

# Task - 3

In [266]:
loaded_model = tf.keras.models.load_model('POS_Stacked_CNN.h5')
y_pred = loaded_model.predict(X_test.toarray())
y_pred = np.argmax(y_pred, axis=1)
y_pred

157/157 [==============================] - 1s 6ms/step


array([0, 0, 5, ..., 7, 7, 4], dtype=int64)

In [267]:
y_pred = le.inverse_transform(y_pred)

In [268]:
actual = d2['label'].to_numpy()

In [269]:
print(actual)
print(y_pred)

['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'instructgpt' 'instructgpt']
['ctrl' 'ctrl' 'grover' ... 'instructgpt' 'instructgpt' 'gpt3']


In [270]:
print(classification_report(actual, y_pred, labels=['ctrl','fair','gpt','gpt2','grover','instructgpt','pplm','xlm','xlnet','gpt3', 'instructgpt']))

              precision    recall  f1-score   support

        ctrl       0.97      0.99      0.98       500
        fair       0.68      0.43      0.53       500
         gpt       1.00      0.97      0.98       500
        gpt2       0.64      0.85      0.73       500
      grover       0.71      0.81      0.75       500
 instructgpt       0.74      0.73      0.74       500
        pplm       0.92      0.77      0.84       500
         xlm       1.00      0.98      0.99       500
       xlnet       1.00      0.97      0.98       500
        gpt3       0.73      0.76      0.74       500
 instructgpt       0.74      0.73      0.74       500

   micro avg       0.83      0.82      0.82      5500
   macro avg       0.83      0.82      0.82      5500
weighted avg       0.83      0.82      0.82      5500



# POS + CNN-LSTM

In [136]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(df['Features'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [137]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['label'])

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [120]:
max_len = X_train_tfidf.shape[1]
input_shape = (max_len,)

In [139]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=input_shape),
    tf.keras.layers.Reshape((max_len, 1)),
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

In [140]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [143]:
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_val.toarray(), y_val))

Epoch 1/10
235/235 [==============================] - 5s 22ms/step - loss: 0.8346 - accuracy: 0.6906 - val_loss: 0.9967 - val_accuracy: 0.6343
Epoch 2/10
235/235 [==============================] - 5s 22ms/step - loss: 0.8359 - accuracy: 0.6867 - val_loss: 0.9026 - val_accuracy: 0.6743
Epoch 3/10
235/235 [==============================] - 5s 22ms/step - loss: 0.8164 - accuracy: 0.6955 - val_loss: 0.9016 - val_accuracy: 0.6652
Epoch 4/10
235/235 [==============================] - 5s 21ms/step - loss: 0.8212 - accuracy: 0.6887 - val_loss: 0.9204 - val_accuracy: 0.6557
Epoch 5/10
235/235 [==============================] - 5s 22ms/step - loss: 0.8104 - accuracy: 0.6955 - val_loss: 0.8692 - val_accuracy: 0.6839
Epoch 6/10
235/235 [==============================] - 5s 21ms/step - loss: 0.7862 - accuracy: 0.7030 - val_loss: 0.8565 - val_accuracy: 0.6818
Epoch 7/10
235/235 [==============================] - 5s 22ms/step - loss: 0.7935 - accuracy: 0.7008 - val_loss: 0.9179 - val_accuracy: 0.6663

In [144]:
y_pred = model.predict(X_test.toarray())

74/74 [==============================] - 8s 8ms/step


In [145]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([ 7,  4,  4, ...,  1, 10,  4], dtype=int64)

In [146]:
y_test

array([ 6,  5,  4, ...,  3, 10,  4])

In [147]:
print(confusion_matrix(y_test, y_pred))

[[207   0   1   0   0   0   0   0   0   1   5]
 [  1  63   0  67  15  44   5   8   3   3   0]
 [  1   0 209   0   0   3   4   0   0   0   0]
 [  0  30   0 140  23   7   0   7   2   4   0]
 [  0   2   1   8 156   2  10  31   5   2   0]
 [  1  41   0  15  11 137   0  13   2   0   2]
 [  1   3   2   5  59  11 119  10   9   1   0]
 [  1   1   0  19  99   7   7  62   4   1   1]
 [  0   4   0   1  70   5  13   6 107   0   0]
 [  3   0   0   0   0   0   0   0   6 206   0]
 [ 14   0   2   0   0   0   0   1   1   9 184]]


In [148]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

        ctrl       0.90      0.97      0.93       214
        fair       0.44      0.30      0.36       209
         gpt       0.97      0.96      0.97       217
        gpt2       0.55      0.66      0.60       213
        gpt3       0.36      0.72      0.48       217
      grover       0.63      0.62      0.63       222
       human       0.75      0.54      0.63       220
 instructgpt       0.45      0.31      0.36       202
        pplm       0.77      0.52      0.62       206
         xlm       0.91      0.96      0.93       215
       xlnet       0.96      0.87      0.91       211

    accuracy                           0.68      2346
   macro avg       0.70      0.67      0.67      2346
weighted avg       0.70      0.68      0.68      2346



In [149]:
model.save('POS_CNN_LSTM.h5')

# Task - 3

In [271]:
loaded_model = tf.keras.models.load_model('POS_CNN_LSTM.h5')
y_pred = loaded_model.predict(X_test.toarray())
y_pred = np.argmax(y_pred, axis=1)
y_pred

157/157 [==============================] - 1s 4ms/step


array([0, 0, 9, ..., 4, 4, 9], dtype=int64)

In [272]:
y_pred = le.inverse_transform(y_pred)

In [273]:
actual = d2['label'].to_numpy()

In [274]:
print(actual)
print(y_pred)

['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'instructgpt' 'instructgpt']
['ctrl' 'ctrl' 'xlm' ... 'gpt3' 'gpt3' 'xlm']


In [275]:
print(classification_report(actual, y_pred, labels=['ctrl','fair','gpt','gpt2','grover','instructgpt','pplm','xlm','xlnet','gpt3', 'instructgpt']))

              precision    recall  f1-score   support

        ctrl       0.95      0.96      0.95       500
        fair       0.47      0.33      0.39       500
         gpt       0.97      0.98      0.97       500
        gpt2       0.59      0.70      0.64       500
      grover       0.62      0.60      0.61       500
 instructgpt       0.52      0.35      0.42       500
        pplm       0.86      0.47      0.61       500
         xlm       0.92      0.96      0.94       500
       xlnet       0.96      0.91      0.93       500
        gpt3       0.41      0.72      0.52       500
 instructgpt       0.52      0.35      0.42       500

   micro avg       0.70      0.66      0.68      5500
   macro avg       0.71      0.66      0.67      5500
weighted avg       0.71      0.66      0.67      5500



# POS + Parallel _ CNN

In [150]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(df['Features'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [151]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['label'])

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [158]:
max_len = X_train_tfidf.shape[1]
input_shape = (max_len,)

In [159]:
num_filters = 64
kernel_sizes = [3, 4, 5]

In [160]:
input_layer = tf.keras.layers.Input(shape=input_shape)
reshape_layer = tf.keras.layers.Reshape((max_len, 1))(input_layer)
conv_layers = []
for kernel_size in kernel_sizes:
    conv_layer = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')(reshape_layer)
    pool_layer = tf.keras.layers.MaxPooling1D(pool_size=max_len - kernel_size + 1)(conv_layer)
    flatten_layer = tf.keras.layers.Flatten()(pool_layer)
    conv_layers.append(flatten_layer)
concat_layer = tf.keras.layers.concatenate(conv_layers, axis=-1)
output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(concat_layer)

In [161]:
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [162]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [165]:
model.fit(X_train.toarray(), y_train, epochs=20, batch_size=32, validation_data=(X_val.toarray(), y_val))

Epoch 1/20
235/235 [==============================] - 1s 2ms/step - loss: 1.0627 - accuracy: 0.6303 - val_loss: 1.1329 - val_accuracy: 0.6157
Epoch 2/20
235/235 [==============================] - 1s 3ms/step - loss: 1.0569 - accuracy: 0.6302 - val_loss: 1.1139 - val_accuracy: 0.6162
Epoch 3/20
235/235 [==============================] - 1s 2ms/step - loss: 1.0505 - accuracy: 0.6362 - val_loss: 1.1361 - val_accuracy: 0.5954
Epoch 4/20
235/235 [==============================] - 0s 2ms/step - loss: 1.0464 - accuracy: 0.6361 - val_loss: 1.1144 - val_accuracy: 0.6039
Epoch 5/20
235/235 [==============================] - 1s 3ms/step - loss: 1.0429 - accuracy: 0.6399 - val_loss: 1.1001 - val_accuracy: 0.6226
Epoch 6/20
235/235 [==============================] - 0s 2ms/step - loss: 1.0379 - accuracy: 0.6382 - val_loss: 1.0996 - val_accuracy: 0.6183
Epoch 7/20
235/235 [==============================] - 0s 2ms/step - loss: 1.0367 - accuracy: 0.6410 - val_loss: 1.0966 - val_accuracy: 0.6167
Epoch 

In [166]:
y_pred = model.predict(X_test.toarray())

74/74 [==============================] - 0s 850us/step


In [167]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([ 7,  4,  4, ...,  1, 10,  7], dtype=int64)

In [168]:
y_test

array([ 6,  5,  4, ...,  3, 10,  4])

In [169]:
print(confusion_matrix(y_test, y_pred))

[[191   0   4   1   1   0   2   0   1   6   8]
 [  5  78   1  39   9  44  11   9   9   3   1]
 [  2   0 213   0   0   0   2   0   0   0   0]
 [  0  40   0 121  14  20   2  10   5   1   0]
 [  3   1   1  17 105   0  32  37  20   1   0]
 [  1  42   4  16   4 125   9  11   4   4   2]
 [  3   0   9   5  35   2 135  13  17   0   1]
 [  1  12   0  15  53   3  18  80  16   4   0]
 [  1   5   0   4  12   0  44   4 128   8   0]
 [  4   2   0   5   2   0   1   0   8 183  10]
 [ 16   0   1   1   0   1   2   0   1  14 175]]


In [170]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

        ctrl       0.84      0.89      0.87       214
        fair       0.43      0.37      0.40       209
         gpt       0.91      0.98      0.95       217
        gpt2       0.54      0.57      0.55       213
        gpt3       0.45      0.48      0.46       217
      grover       0.64      0.56      0.60       222
       human       0.52      0.61      0.56       220
 instructgpt       0.49      0.40      0.44       202
        pplm       0.61      0.62      0.62       206
         xlm       0.82      0.85      0.83       215
       xlnet       0.89      0.83      0.86       211

    accuracy                           0.65      2346
   macro avg       0.65      0.65      0.65      2346
weighted avg       0.65      0.65      0.65      2346



In [171]:
model.save('POS_Parallel_CNN.h5')

# Task - 3

In [276]:
loaded_model = tf.keras.models.load_model('POS_Parallel_CNN.h5')
y_pred = loaded_model.predict(X_test.toarray())
y_pred = np.argmax(y_pred, axis=1)
y_pred

157/157 [==============================] - 0s 869us/step


array([0, 0, 8, ..., 7, 3, 4], dtype=int64)

In [277]:
y_pred = le.inverse_transform(y_pred)

In [278]:
actual = d2['label'].to_numpy()

In [279]:
print(actual)
print(y_pred)

['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'instructgpt' 'instructgpt']
['ctrl' 'ctrl' 'pplm' ... 'instructgpt' 'gpt2' 'gpt3']


In [280]:
print(classification_report(actual, y_pred, labels=['ctrl','fair','gpt','gpt2','grover','instructgpt','pplm','xlm','xlnet','gpt3', 'instructgpt']))

              precision    recall  f1-score   support

        ctrl       0.86      0.91      0.88       500
        fair       0.45      0.38      0.41       500
         gpt       0.94      0.97      0.96       500
        gpt2       0.56      0.54      0.55       500
      grover       0.61      0.60      0.61       500
 instructgpt       0.53      0.44      0.48       500
        pplm       0.71      0.65      0.68       500
         xlm       0.84      0.85      0.85       500
       xlnet       0.91      0.80      0.85       500
        gpt3       0.50      0.46      0.48       500
 instructgpt       0.53      0.44      0.48       500

   micro avg       0.68      0.64      0.66      5500
   macro avg       0.68      0.64      0.66      5500
weighted avg       0.68      0.64      0.66      5500



# POS + XGBoost

In [354]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(df['Features'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [355]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['label'])

In [356]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

In [357]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9380, 76), (2346, 76), (9380,), (2346,))

In [358]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

In [359]:
xgb.fit(X_train.toarray(), y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [360]:
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score

In [361]:
y_pred = xgb.predict(X_test.toarray())

In [362]:
y_pred = le.inverse_transform(y_pred)
y_pred

array(['gpt3', 'grover', 'gpt3', ..., 'gpt2', 'xlnet', 'gpt3'],
      dtype=object)

In [363]:
y_test = le.inverse_transform(y_test)

In [364]:
y_test

array(['human', 'grover', 'gpt3', ..., 'gpt2', 'xlnet', 'gpt3'],
      dtype=object)

In [365]:
print(accuracy_score(y_test, y_pred))

0.840153452685422


In [366]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        ctrl       0.97      1.00      0.98       214
        fair       0.67      0.65      0.66       209
         gpt       0.98      0.98      0.98       217
        gpt2       0.80      0.77      0.79       213
        gpt3       0.66      0.67      0.67       217
      grover       0.83      0.80      0.82       222
       human       0.81      0.75      0.78       220
 instructgpt       0.70      0.73      0.72       202
        pplm       0.84      0.92      0.88       206
         xlm       0.99      1.00      0.99       215
       xlnet       0.99      0.98      0.98       211

    accuracy                           0.84      2346
   macro avg       0.84      0.84      0.84      2346
weighted avg       0.84      0.84      0.84      2346



# Task 3

In [368]:
X_test_task_3 = vectorizer.transform(T1['Features'])
X_test_task_3 = tfidf_transformer.transform(X_test_task_3)

In [371]:
y_pred = xgb.predict(X_test_task_3.toarray())
# y_pred = np.argmax(y_pred, axis=1)
# y_pred

In [373]:
y_pred = le.inverse_transform(y_pred)

In [374]:
actual = d2['label'].to_numpy()

In [375]:
print(actual)
print(y_pred)

['ctrl' 'ctrl' 'ctrl' ... 'instructgpt' 'instructgpt' 'instructgpt']
['ctrl' 'ctrl' 'ctrl' ... 'gpt3' 'instructgpt' 'instructgpt']


In [376]:
print(classification_report(actual, y_pred, labels=['ctrl','fair','gpt','gpt2','grover','instructgpt','pplm','xlm','xlnet','gpt3', 'instructgpt']))

              precision    recall  f1-score   support

        ctrl       0.99      1.00      1.00       500
        fair       0.94      0.94      0.94       500
         gpt       1.00      1.00      1.00       500
        gpt2       0.96      0.96      0.96       500
      grover       0.97      0.96      0.96       500
 instructgpt       0.95      0.94      0.94       500
        pplm       0.98      0.98      0.98       500
         xlm       1.00      1.00      1.00       500
       xlnet       1.00      0.99      0.99       500
        gpt3       0.96      0.93      0.94       500
 instructgpt       0.95      0.94      0.94       500

   micro avg       0.97      0.97      0.97      5500
   macro avg       0.97      0.97      0.97      5500
weighted avg       0.97      0.97      0.97      5500

